In [64]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
from datetime import datetime

In [91]:
courses = pd.read_csv('../data/processed/courses.csv')
reviews = pd.read_csv('../data/processed/reviews.csv')
leads = pd.read_csv('../data/processed/leads.csv')
enrollments = pd.read_csv('../data/processed/enrollments.csv')

## Courses
This is de course dataframe schema

In [129]:
courses_schema = pd.read_csv('../data/processed/courses_schema.csv')

In [130]:
courses_schema

,column_name,description,type,comments
0,course_id,The unique identifier of the course,string,For reasons of security they have been encripted
1,title,Course title,string,
2,description,Course description,string,
3,syllabus,Course syllabus,string,Contains HTML tags
4,course_type,Course type id,numeric,"For example: Master's degree, Bachelor's degre..."
5,category,Course category id,numeric,Course topic
6,methodology,"Course methodology id (online, classroom based...",numeric,"1 = Classroom, 2 = Distance, 3 = Online, 8 = C..."
7,price_range,Maximum and minimum price of the course in euros,numeric,From 0 = Free to 23 = > 9000 €. -1 for courses...
8,duration,Maximum course duration,numeric,"10 = hours, 20 = days, 30 = weeks, 40 = 3 mo..."
9,flexible,Whether or not it is a self paced course,boolean,"1 = yes, 0 = no"


### Duplicated rows

In [94]:
courses[courses.duplicated(subset='course_id')]

,course_id,title,description,syllabus,methodology,category,course_type,price_range,duration,flexible,language,location,updated_on


There are no duplicated rows in this data frame

### Missing data

In [95]:
courses.head()

,course_id,title,description,syllabus,methodology,category,course_type,price_range,duration,flexible,language,location,updated_on
0,291f3500fcc8230cbdc52f59555f0cb6,Máster en Gestión de Proyectos Logísticos SAP,Emagister.com pone a tu disposición el Máster ...,<br /><strong>PROGRAMA:</strong><br /><br /><b...,1,805,1230,22,60,0,es,35.0,2019-12-03 14:09:30.0
1,3ea3378c14939797fcbcf29f2fa9bb29,Máster Universitario en Producción y Realizaci...,Emagister.com pone a tu disposición el Máster ...,<strong>Temario</strong><br /><br />M&oacute;d...,1,14,1235,20,-1,0,es,35.0,2019-12-03 14:07:31.0
2,18268a14024efd522e0b92a75b7c46e7,Curso de SEO Y SEM ( POSICIONAMIENTO EN LAS RE...,Emagister.com pone a tu disposición el curso d...,<p>Posicionamiento Natural en Buscadores SEO:<...,1,597,1000,14,30,0,es,4.0,2019-12-03 14:05:46.0
3,a9522a489bea6a940ee4b8370c13b06e,Curso de Especialista en Periodismo y Moda,Emagister.com pone a tu disposición el Curso d...,<strong>Temario</strong><br /><br /><br /><str...,1,416,1000,16,50,0,es,35.0,2019-12-03 14:03:31.0
4,8e49c7e1e0edce52656762fe7480b430,Máster Universitario en Abogacía (Acceso y Eje...,¿Tienes estudios de Derecho y quieres ejercer?...,<strong>I. Marco Jur&iacute;dico Colegial y De...,1,5,1235,23,-1,0,es,35.0,2019-12-03 14:02:03.0


In [96]:
courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41533 entries, 0 to 41532
Data columns (total 13 columns):
course_id      41533 non-null object
title          41533 non-null object
description    41283 non-null object
syllabus       41529 non-null object
methodology    41533 non-null int64
category       41533 non-null int64
course_type    41533 non-null int64
price_range    41533 non-null int64
duration       41533 non-null int64
flexible       41533 non-null int64
language       41533 non-null object
location       13313 non-null float64
updated_on     41533 non-null object
dtypes: float64(1), int64(6), object(6)
memory usage: 4.1+ MB


#### `syllabus` and `description` columns
There are courses without description and syllabus, in this case this is not a problem because I will merge title, description and syllabus columns in one column.
#### `location` column

NaN values and -1 in `location` column should correspond to courses with Online (3) or Distance (2) methodology

In [105]:
# Missing and -1 values in location column
nan_location = np.isnan(courses['location'])
neg_location = courses['location'] == -1
courses[nan_location | neg_location]['methodology'].value_counts()

3    19471
2     8301
Name: methodology, dtype: int64

Courses without location and a face-to-face methodology (1, 8 and 9) are those that have more than one location. To prevent confusions, I will replace these values with 0.

In [106]:
face_to_face_courses = courses['methodology'].isin([1, 8, 9])
courses.loc[(nan_location | neg_location) & face_to_face_courses, 'location'] = 0.0

In [107]:
courses[(nan_location | neg_location) & face_to_face_courses]['methodology'].value_counts()

Series([], Name: methodology, dtype: int64)

In [109]:
courses[courses['location'] == 0.0]['methodology'].value_counts()

8    366
9     80
1     33
Name: methodology, dtype: int64

Online and distance courses with `location` are courses that were classroom based in the past and when they were converted to the new methodology, the location was not updated. Now, I will replace those locations with NaN

In [110]:
distance_courses = courses['methodology'] == 2
online_courses = courses['methodology'] == 3
courses[distance_courses | online_courses]['location'].value_counts()

35.0      14
36.0      11
9.0        4
45.0       4
192.0      3
26.0       2
13.0       2
12.0       2
51.0       2
21.0       2
6.0        1
64.0       1
171.0      1
122.0      1
1102.0     1
65.0       1
185.0      1
1662.0     1
19.0       1
55.0       1
20.0       1
5.0        1
66.0       1
266.0      1
42.0       1
392.0      1
95.0       1
492.0      1
31.0       1
39.0       1
Name: location, dtype: int64

In [111]:
courses.loc[distance_courses | online_courses, 'location'] = np.nan

In [112]:
courses[distance_courses | online_courses]['location'].value_counts()

Series([], Name: location, dtype: int64)

Finally, lets normalize those values. I will set region -1 to courses whose methodology is Distance (2) and Online (3)

In [117]:
courses.loc[distance_courses | online_courses, 'location'] = -1.0

In [121]:
courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41533 entries, 0 to 41532
Data columns (total 13 columns):
course_id      41533 non-null object
title          41533 non-null object
description    41283 non-null object
syllabus       41529 non-null object
methodology    41533 non-null int64
category       41533 non-null int64
course_type    41533 non-null int64
price_range    41533 non-null int64
duration       41533 non-null int64
flexible       41533 non-null int64
language       41533 non-null object
location       41533 non-null float64
updated_on     41533 non-null object
dtypes: float64(1), int64(6), object(6)
memory usage: 4.1+ MB


### Data types
I will change `location` column to int and `updated_on` to datetime

In [126]:
courses['location'] = courses['location'].astype(int)
courses['updated_on'] = pd.to_datetime(courses['updated_on'])

In [128]:
courses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41533 entries, 0 to 41532
Data columns (total 13 columns):
course_id      41533 non-null object
title          41533 non-null object
description    41283 non-null object
syllabus       41529 non-null object
methodology    41533 non-null int64
category       41533 non-null int64
course_type    41533 non-null int64
price_range    41533 non-null int64
duration       41533 non-null int64
flexible       41533 non-null int64
language       41533 non-null object
location       41533 non-null int64
updated_on     41533 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(7), object(5)
memory usage: 4.1+ MB


### Clean text columns

In [134]:
def clean_syllabus(syllabus):
    return ' '.join(bs(syllabus, 'lxml').get_text(' ', strip=True).split()).replace(u'\xa0', ' ') if not pd.isnull(syllabus) else None

def clean_description(description):
    return description.replace(u'\xa0', ' ') if not pd.isnull(description) else None

In [132]:
# remove HTML tags from syllabus
courses['syllabus'] = courses['syllabus'].apply(clean_syllabus)

In [135]:
# clean unicode characters from description
courses['description'] = courses['description'].apply(clean_description)

In [136]:
courses.head()

,course_id,title,description,syllabus,methodology,category,course_type,price_range,duration,flexible,language,location,updated_on
0,291f3500fcc8230cbdc52f59555f0cb6,Máster en Gestión de Proyectos Logísticos SAP,Emagister.com pone a tu disposición el Máster ...,PROGRAMA: - Módulo de Introducción a Procesos ...,1,805,1230,22,60,0,es,35,2019-12-03 14:09:30
1,3ea3378c14939797fcbcf29f2fa9bb29,Máster Universitario en Producción y Realizaci...,Emagister.com pone a tu disposición el Máster ...,Temario Módulo I: FUNDAMENTOS DE LA REALIZACIÓ...,1,14,1235,20,-1,0,es,35,2019-12-03 14:07:31
2,18268a14024efd522e0b92a75b7c46e7,Curso de SEO Y SEM ( POSICIONAMIENTO EN LAS RE...,Emagister.com pone a tu disposición el curso d...,Posicionamiento Natural en Buscadores SEO: Int...,1,597,1000,14,30,0,es,4,2019-12-03 14:05:46
3,a9522a489bea6a940ee4b8370c13b06e,Curso de Especialista en Periodismo y Moda,Emagister.com pone a tu disposición el Curso d...,Temario Módulo I : Sociología de la Moda Módul...,1,416,1000,16,50,0,es,35,2019-12-03 14:03:31
4,8e49c7e1e0edce52656762fe7480b430,Máster Universitario en Abogacía (Acceso y Eje...,¿Tienes estudios de Derecho y quieres ejercer?...,I. Marco Jurídico Colegial y Deontología Profe...,1,5,1235,23,-1,0,es,35,2019-12-03 14:02:03


In [137]:
courses.to_csv('../data/clean/courses.csv', index=False)

## Reviews

In [138]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 5 columns):
user_id            40000 non-null object
course_id          40000 non-null object
rating             40000 non-null int64
would_recommend    40000 non-null int64
created_on         40000 non-null object
dtypes: int64(2), object(3)
memory usage: 1.5+ MB


This dataframe has no null values.

### Data types
I will convert the `created_on` column to datetime format

In [144]:
reviews['created_on'] = pd.to_datetime(reviews['created_on'])

In [145]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 5 columns):
user_id            40000 non-null object
course_id          40000 non-null object
rating             40000 non-null int64
would_recommend    40000 non-null int64
created_on         40000 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 1.5+ MB


In [146]:
reviews.to_csv('../data/clean/reviews.csv', index=False)

## Enrollments

In [139]:
enrollments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36473 entries, 0 to 36472
Data columns (total 3 columns):
user_id       36473 non-null object
course_id     36473 non-null object
created_on    36473 non-null object
dtypes: object(3)
memory usage: 854.9+ KB


This dataframe has no null values.

### Data types
I will convert the `created_on` column to datetime format

In [142]:
enrollments['created_on'] = pd.to_datetime(enrollments['created_on'])

In [143]:
enrollments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36473 entries, 0 to 36472
Data columns (total 3 columns):
user_id       36473 non-null object
course_id     36473 non-null object
created_on    36473 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 854.9+ KB


In [147]:
enrollments.to_csv('../data/clean/enrollments.csv', index=False)

## Leads

In [140]:
leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
user_id      50000 non-null object
course_id    50000 non-null object
dtypes: object(2)
memory usage: 781.3+ KB


In [141]:
leads.head()

,user_id,course_id
0,10fb910da113efc5cd74b81065400e12,e0c424c5fc399cdbb1bf8549bde633c5
1,f43f477510f8c2048e600a85ab339369,f8ddab8ef084cc588c984b58767c5cbb
2,c7011ab2cb1b9a24e59e5ba4ba217d64,3395fa729651491cd1a84502ed090327
3,8596ef53d30e7baf338fe6854bc0a443,4e0abe5d6fd932aea04bdbd79360bb4c
4,f88b4e58210523ee2351dcf7212781bc,12551267b905d157247a6618809a8e50


No missing values found and no data conversion is needed in this data set

In [148]:
leads.to_csv('../data/clean/leads.csv', index=False)